In [ ]:
import os

import matplotlib.pyplot as plt

import bluemira.codes.process as process
from bluemira.base.config import Configuration
from bluemira.base.file import get_bluemira_path
from bluemira.base.parameter import ParameterFrame
from bluemira.equilibria import Equilibrium
from bluemira.geometry._deprecated_loop import Loop
from bluemira.radiation_transport.advective_transport import ChargedParticleSolver
from bluemira.radiation_transport.radiation_profile import STCore, STScrapeOffLayer

Read equilibrium

In [ ]:
read_path = get_bluemira_path("equilibria", subfolder="data")
eq_name = "DN-DEMO_eqref.json"
eq_name = os.sep.join([read_path, eq_name])
eq = Equilibrium.from_eqdsk(eq_name, load_large_file=True)

Get first wall shape

In [ ]:
read_path = get_bluemira_path("radiation_transport/test_data", subfolder="tests")
fw_name = "DN_fw_shape.json"
fw_name = os.sep.join([read_path, fw_name])
fw_shape = Loop.from_file(fw_name)

Run particle solver

In [ ]:
p_solver_params = ParameterFrame()
solver = ChargedParticleSolver(p_solver_params, eq, dx_mp=0.001)
solver.analyse(first_wall=fw_shape)

Run PROCESS solver

In [ ]:
PROCESS_PATH = ""
binary = f"{PROCESS_PATH}process"

new_params = {
    "kappa": 1.6969830041844367,
}
params = Configuration(new_params)

build_config = {
    "mode": "run",
    "binary": binary,
}

process_solver = process.Solver(
    params=params,
    build_config=build_config,
)
process_solver.run()

Get impurity fractions

In [ ]:
impurity_content = {
    "H": process_solver.get_species_fraction("H"),
    "He": process_solver.get_species_fraction("He"),
    "Xe": process_solver.get_species_fraction("Xe"),
    "W": process_solver.get_species_fraction("W"),
}

Get impurity data: temperature reference
and radiative loss function reference

In [ ]:
impurity_data = {
    "H": {
        "T_ref": process_solver.get_species_data("H")[0],
        "L_ref": process_solver.get_species_data("H")[1],
    },
    "He": {
        "T_ref": process_solver.get_species_data("He")[0],
        "L_ref": process_solver.get_species_data("He")[1],
    },
    "Xe": {
        "T_ref": process_solver.get_species_data("Xe")[0],
        "L_ref": process_solver.get_species_data("Xe")[1],
    },
    "W": {
        "T_ref": process_solver.get_species_data("W")[0],
        "L_ref": process_solver.get_species_data("W")[1],
    },
}

Eventually customise plasma parameters to deviate from default values

In [ ]:
plasma_params = ParameterFrame(
    # fmt: off
    [
        ["kappa", "Elongation", 3, "dimensionless", None, "Input"],
    ]
    # fmt: on
)

Eventually customise radiation parameters to deviate from default values

In [ ]:
rad_params = ParameterFrame(
    # fmt: off
    [
        ["p_sol", "power entering the SoL", 300e6, "W", None, "Input"],
    ]
    # fmt: on
)

Run core radiation source calculation for Spherical Tokamak

In [ ]:
stcore = STCore(solver, impurity_content, impurity_data, plasma_params, rad_params)

Build radiation profile at the midplane and 2D core radiation map

In [ ]:
stcore.build_mp_rad_profile()
stcore.build_core_radiation_map()

Run scrape-off layer radiation source calculation for Spherical Tokamak

In [ ]:
stsol = STScrapeOffLayer(
    solver, impurity_content, impurity_data, plasma_params, rad_params, fw_shape
)

Build temperature and density poloidal profiles for the scrape-off layer.
The output contains for lists for the four sectors: LFS lower divertor,
LFS upper divertor, HFS lower divertor, HFS upper divertor

In [ ]:
t_and_n_sol_profiles = stsol.build_sol_profiles(fw_shape)

Build radiation source profiles for the scrape-off layer.
The output contains for lists for the four sectors.

In [ ]:
rad_sector_profiles = stsol.build_sol_rad_distribution(*t_and_n_sol_profiles)

Build 2D scrape-off layer radiation map

In [ ]:
stsol.build_sol_radiation_map(*rad_sector_profiles, fw_shape)

Plot results

In [ ]:
plt.show()